# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

607

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="
query_url

'http://api.openweathermap.org/data/2.5/weather?appid=f2cb334aee0d8f1c4aaa53f4e4f9bd89&units=imperial&q='

In [10]:
# set up the lists to hold reponse info
lat = []
temp = [] 
humidity = []
clouds = []
wind = []
temp_max = []

#Variables to keep track of response number
record_num = 1
record_set = 1

# Loop through the list of cities and perform a request for data on each
for city in cities:
    try:
        response = requests.get(query_url + city).json()
        lat.append(response['coord']['lat'])
        temp.append(response['main']['temp'])
        humidity.append(response['main']['humidity'])
        clouds.append(response['clouds']['all'])
        wind.append(response['wind']['speed'])
        record_num += 1
    except Exception:
        print("City not found")
    #I've borrowed this piece of the code from internet wisdom; it turned out that Weather API has limitation
    #on calls per time. it seems to be working better with this limitation!
    if record_num == 51:
        record_set += 1
        record_num = 1
        time.sleep(60)

City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found
City not found


In [12]:
# create a data frame from the received inputs
weather_dict = {
    "city": cities,
    "lat": lat,
    "temp": temp,
    "humidity": humidity,
    "clouds": clouds,
    "wind_speed": wind
}
weather_data_df = pd.DataFrame.from_dict(weather_dict, orient='index')
weather_data_df.transpose()
weather_data_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,597,598,599,600,601,602,603,604,605,606
city,sao joao da barra,cape town,tuktoyaktuk,rikitea,norman wells,inuvik,anadyr,wuwei,belushya guba,mataura,...,alofi,barbar,roald,wawa,skalistyy,fairlie,srisailam,brigantine,bolungarvik,rakovnik
lat,-21.64,-33.93,69.45,-23.12,65.28,68.35,64.75,37.93,-46.19,-22.67,...,None,None,None,None,None,None,None,None,None,None
temp,71.4,59.43,37.4,73.94,60.8,48.2,62.6,71.01,39,69.08,...,None,None,None,None,None,None,None,None,None,None
humidity,87,87,100,80,93,76,42,56,83,63,...,None,None,None,None,None,None,None,None,None,None
clouds,0,40,90,77,90,75,40,100,1,4,...,None,None,None,None,None,None,None,None,None,None
wind_speed,9.66,12.75,9.17,13.76,6.93,8.05,13.42,15.73,3,12.03,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
#drop missing data
weather_data_clean_df = weather_data_df.dropna(axis='columns')
weather_data_fin_df = weather_data_clean_df.transpose()
weather_data_fin_df.head()

In [ ]:
weather_data_fin_df.info()

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
weather_data_fin_df.to_csv (r'C:\Users\spalt\OneDrive\Desktop\Homework\export_weath_dataframe.csv', index = False, header=True)

In [ ]:
print (weather_data_fin_df)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
# Build a scatter plot Lat vs Temp
plt.scatter(weather_data_df["lat"], weather_data_df["temp"], marker="o")

# Incorporate the other graph properties
plt.title("Latitude vs. Temperature Plot")
plt.ylabel("Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("LatitudeVsTemperaturePlot.png")

# Show plot
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
# Build a scatter plot Lat vs Temp
plt.scatter(weather_data_df["lat"], weather_data_df["humidity"], marker="o")

# Incorporate the other graph properties
plt.title("Latitude vs. Humidity Plot")
plt.ylabel("Humidity, %")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("LatitudeVsHumidityPlot.png")

# Show plot
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
# Build a scatter plot Lat vs Temp
plt.scatter(weather_data_df["lat"], weather_data_df["clouds"], marker="o")

# Incorporate the other graph properties
plt.title("Latitude vs. Cloudiness Plot")
plt.ylabel("Cloudiness")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("LatitudeVsCloudinessPlot.png")

# Show plot
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
# Build a scatter plot Lat vs Temp
plt.scatter(weather_data_df["lat"], weather_data_df["wind_speed"], marker="o")

# Incorporate the other graph properties
plt.title("Latitude vs. Wind Speed Plot")
plt.ylabel("Wind Speed")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("LatitudeVsWindPlot.png")

# Show plot
plt.show()

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames
#New Hemispher
lat_nh = []
temp_nh = [] 
humidity_nh = []
clouds_nh = []
wind_nh = []
nh = "Northern Hemisphere"
response_nh = requests.get(query_url + nh).json()
lat_nh.append(response['coord']['lat'])
temp_nh.append(response['main']['temp'])
humidity_nh.append(response['main']['humidity'])
clouds_nh.append(response['clouds']['all'])
wind_nh.append(response['wind']['speed'])

# print(f"The latitude information received is: {lat}")
# print(f"The temperature information received is: {temp}")
# print(f"The temperature information received is: {humidity}")
# print(f"The latitude information received is: {clouds}")
# print(f"The latitude information received is: {wind}")

In [ ]:
# Create a Scatter Plot for temperature vs latitude
x_values = weather_data_nh['lat']
y_values = weather_data_nh['temp']
plt.scatter(x_values,y_values)
plt.xlabel('Latitude')
plt.ylabel('Temperature')
plt.show()

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression